In [13]:
import requests
 
link = "https://epsg.io/28350.prettywkt"
f = requests.get(link)
lines=f.text.split(',')
sections=lines[0].split('"')
print(sections[1].replace(' zone ',''))

GDA94 / MGA50


In [25]:
from math import sqrt, pow, acos, degrees
###########################################
# Apical angle between tree points, first point is at apex
###########################################
def tri_angle(p1x,p1y,p2x,p2y,p3x,p3y):
    p12=sqrt(pow(p1x-p2x,2.0)+pow(p1y-p2y,2.0))
    p13=sqrt(pow(p1x-p3x,2.0)+pow(p1y-p3y,2.0))
    p23=sqrt(pow(p2x-p3x,2.0)+pow(p2y-p3y,2.0))
    numerator=pow(p12,2.0)+pow(p13,2.0)-pow(p23,2.0)
    divisor=2*p12*p13
    angle=degrees(acos(numerator/divisor))
    return(angle)


print(tri_angle(1.0,1.0,0.0,0.0,2.1,2.0))

1.4142135623730951 1.4866068747318506 2.9 -4.199999999999999 4.204759208325728 3.094009550312804
177.2736890060934


In [2]:
def get_dtm(path_out, minlong,maxlong,minlat,maxlat):


    bbox=(minlong,minlat,maxlong,maxlat)

    url="http://services.ga.gov.au/gis/services/DEM_SRTM_1Second_over_Bathymetry_Topography/MapServer/WCSServer?"
    wcs = WebCoverageService(url,version='1.0.0')

    cvg=wcs.getCoverage(identifier='1',  bbox=bbox, format='GeoTIFF', crs=4326, width=200, height=200)

    f = open(path_out, 'wb')
    bytes_written = f.write(cvg.read())
    f.close()
    print("dtm geotif saved as",path_out)
    


In [6]:
from shapely.geometry import Polygon
import geopandas as gpd
from owslib.wcs import WebCoverageService

minx=500057  #region of interest coordinates in metre-based system (or non-degree system)
maxx=603028
miny=7455348
maxy=7567953
model_top=1200
model_base=-8200

#PATHS

test_data_path='../test_data3/'
geology_file='hams2_geol.shp'   #input geology file (if local)
fault_file='GEOS_GEOLOGY_LINEARSTRUCTURE_500K_GSD.shp' #input fault file (if local)
structure_file='hams2_structure.shp' #input bedding orientation file (if local)
m2m_cpp_path='../m2m_cpp/'


#CRS

src_crs = {'init': 'EPSG:4326'}  # coordinate reference system for imported dtms (geodetic lat/long WGS84)
dst_crs = {'init': 'EPSG:28350'} # coordinate system for data
lat_point_list = [miny, miny, maxy, maxy, maxy]
lon_point_list = [minx, maxx, maxx, minx, minx]
bbox_geom = Polygon(zip(lon_point_list, lat_point_list))
polygon = gpd.GeoDataFrame(index=[0], crs=dst_crs, geometry=[bbox_geom]) 
bbox=(minx,miny,maxx,maxy)
polygon_ll=polygon.to_crs(src_crs)
step_out=0.045 #add (in degrees) so edge pixel from dtm reprojection are not found

minlong=polygon_ll.total_bounds[0]-step_out
maxlong=polygon_ll.total_bounds[2]+step_out
minlat=polygon_ll.total_bounds[1]-step_out
maxlat=polygon_ll.total_bounds[3]+step_out

In [8]:
get_dtm(test_data_path+'tmp/testdtm.tif', minlong,maxlong,minlat,maxlat)

dtm geotif saved as ../test_data3/tmp/testdtm.tif


In [ ]:
#!/usr/bin/env python

import numpy as num

celltype_map = {'IEEE4ByteReal': num.float32, 'IEEE8ByteReal': num.float64}


def write_ermapper_grid(ofile, data, header = {}):
    """
    write_ermapper_grid(ofile, data, header = {}):
    Function to write a 2D numeric array to an ERMapper grid.  There are a series of conventions adopted within
    this code, specifically:
    1)  The registration coordinate for the data is the SW (or lower-left) corner of the data
    2)  The registration coordinates refer to cell centres
    3)  The data is a 2D numeric array with the NW-most data in element (0,0) and the SE-most data in element (N,M)
        where N is the last line and M is the last column
    4)  There has been no testng of the use of a rotated grid.  Best to keep data in an NS orientation
    Input Parameters:
    ofile:      string - filename for output (note the output will consist of two files
                ofile and ofile.ers.  Either of these can be entered into this function
    data:       array - 2D array containing the data to be output to the grid
    header:     dictionary - contains spatial information about the grid, in particular:
                    header['datum'] datum for the data ('"GDA94"')
                    header['projection'] - either '"GEOGRAPHIC"' or '"PROJECTED"'
                    header['coordinatetype'] - either 'EN' (for eastings/northings) or
                                                      'LL' (for lat/long)
                    header['rotation'] - rotation of grid ('0:0:0.0')
                    header['celltype'] - data type for writing data ('IEEE4ByteReal')
                    header['nullcellvalue'] - value for null cells ('-99999')
                    header['xdimension'] - cell size in x-dir in units dictated by 'coordinatetype' ('100')
                    header['registrationcellx'] == '0'
                    header['ydimension'] - cell size in y-dir in units dictated by 'coordinatetype' ('100')
                    header['longitude'] - co-ordinate of registration cell ('0:0:0')
                    header['latitude'] - co-ordinate of registration line ('0:0:0')
                    header['nrofbands'] - number of bands ('1')
                    header['value'] - name of grid ('"Default_Band"')
                    Some entries are determined automatically from the data
                    header['nroflines'] - number of lines in data
                    header['nrofcellsperline'] - number of columns in data
                    header['registrationcelly'] == last line of data
    Written by Trevor Dhu, Geoscience Australia 2005
    """
    # extract filenames for header and data files from ofile
    ers_index = ofile.find('.ers')
    if ers_index > 0:
        data_file = ofile[0:ers_index]
        header_file = ofile
    else:
        data_file = ofile
        header_file = ofile + '.ers'


    # Check that the data is a 2 dimensional array
    data_size = num.shape(data)
    assert len(data_size) == 2

    header['nroflines'] = str(data_size[0])
    header['nrofcellsperline'] = str(data_size[1])


    header = create_default_header(header)
    write_ermapper_header(header_file, header)
    write_ermapper_data(data, data_file, data_format = header['celltype'])


def read_ermapper_grid(ifile):
    ers_index = ifile.find('.ers')
    if ers_index > 0:
        data_file = ifile[0:ers_index]
        header_file = ifile
    else:
        data_file = ifile
        header_file = ifile + '.ers'

    header = read_ermapper_header(header_file)

    nroflines = int(header['nroflines'])
    nrofcellsperlines = int(header['nrofcellsperline'])
    data = read_ermapper_data(data_file)
    data = num.reshape(data,(nroflines,nrofcellsperlines))
    return data


def write_ermapper_header(ofile, header = {}):

    header = create_default_header(header)
    # Determine if the dataset is in lats/longs or eastings/northings and set header parameters
    # accordingly
    if header['coordinatetype'] == 'LL':
        X_Class = 'Longitude'
        Y_Class = 'Latitude'
    elif header['coordinatetype'] == 'EN':
        X_Class = 'Eastings'
        Y_Class = 'Northings'

    # open the header file for writing to
    fid = open(ofile,'wt')

    # Begin writing the header
    fid.write('DatasetHeader Begin\n')
    fid.write('\tVersion\t\t= "6.4"\n')
    fid.write('\tDatasetType\t= ERStorage\n')
    fid.write('\tDataType\t= Raster\n')
    fid.write('\tByteOrder\t= LSBFirst\n')

    # Write the coordinate space information
    fid.write('\tCoordinateSpace Begin\n')
    fid.write('\t\tDatum\t\t\t = ' + header['datum'] + '\n')
    fid.write('\t\tProjection\t\t = ' + header['projection'] + '\n')
    fid.write('\t\tCoordinateType\t = ' + header['coordinatetype'] + '\n')
    fid.write('\t\tRotation\t\t = ' + header['rotation'] + '\n')
    fid.write('\t\tUnits\t\t = ' + header['units'] + '\n')
    fid.write('\tCoordinateSpace End\n')

    # Write the raster information
    fid.write('\tRasterInfo Begin\n')
    fid.write('\t\tCellType\t\t\t = ' + header['celltype'] + '\n')
    fid.write('\t\tNullCellValue\t\t = ' + header['nullcellvalue'] + '\n')
    fid.write('\t\tRegistrationCellX\t\t = ' + header['registrationcellx'] +'\n')
    fid.write('\t\tRegistrationCellY\t\t = ' + header['registrationcelly'] +'\n')
    # Write the cellsize information
    fid.write('\t\tCellInfo Begin\n')
    fid.write('\t\t\tXDimension\t\t\t = ' + header['xdimension'] + '\n')
    fid.write('\t\t\tYDimension\t\t\t = ' + header['ydimension'] + '\n')
    fid.write('\t\tCellInfo End\n')
    # Continue with wrting the raster information
    fid.write('\t\tNrOfLines\t\t\t = ' + header['nroflines'] + '\n')
    fid.write('\t\tNrOfCellsPerLine\t = ' + header['nrofcellsperline'] + '\n')
    # Write the registration coordinate information
    fid.write('\t\tRegistrationCoord Begin\n')
    fid.write('\t\t\t' + X_Class + '\t\t\t = ' + header[X_Class.lower()] + '\n')
    fid.write('\t\t\t' + Y_Class + '\t\t\t = ' + header[Y_Class.lower()] + '\n')
    fid.write('\t\tRegistrationCoord End\n')
    # Continue with wrting the raster information
    fid.write('\t\tNrOfBands\t\t\t = ' + header['nrofbands'] + '\n')
    fid.write('\t\tBandID Begin\n')
    fid.write('\t\t\tValue\t\t\t\t = ' + header['value'] + '\n')
    fid.write('\t\tBandID End\n')
    fid.write('\tRasterInfo End\n')
    fid.write('DatasetHeader End\n')

    fid.close

def read_ermapper_header(ifile):
    # function for reading an ERMapper header from file
    header = {}

    fid = open(ifile,'rt')
    header_string = fid.readlines()
    fid.close()

    for line in header_string:
        if line.find('=') > 0:
            tmp_string = line.strip().split('=')
            header[tmp_string[0].strip().lower()]= tmp_string[1].strip()

    return header

def write_ermapper_data(grid, ofile, data_format=num.float32):


    try:
        data_format = celltype_map[data_format]
    except:
        pass


    #if isinstance(data_format, basestring):
    #    #celltype_map is defined at top of code
    #    if celltype_map.has_key(data_format):
    #        data_format = celltype_map[data_format]
    #    else:
    #        msg = 'Format %s is not yet defined by celltype_map' %data_format
    #        raise Exception(msg)


    # Convert the array to data_format (default format is Float32)
    grid_as_float = grid.astype(data_format)

    # Convert array to a string for writing to output file
    output_string = grid_as_float.tostring()

    # open output file in a binary format and write the output string
    fid = open(ofile,'wb')
    fid.write(output_string)
    fid.close()


def read_ermapper_data(ifile, data_format = num.float32):
    # open input file in a binary format and read the input string
    fid = open(ifile,'rb')
    input_string = fid.read()
    fid.close()

    # convert input string to required format (Note default format is num.float32)
    grid_as_float = num.frombuffer(input_string,data_format)
    return grid_as_float

def create_default_header(header = {}):
    # fill any blanks in a header dictionary with default values
    # input parameters:
    # header:   a dictionary containing fields that are not meant
    #           to be filled with default values


    if not header.has_key('datum'):
        header['datum'] = '"GDA94"'
    if not header.has_key('projection'):
        header['projection'] = '"GEOGRAPHIC"'
    if not header.has_key('coordinatetype'):
        header['coordinatetype'] = 'LL'
    if not header.has_key('rotation'):
        header['rotation'] = '0:0:0.0'
    if not header.has_key('units'):
        header['units'] = '"METERS"'
    if not header.has_key('celltype'):
        header['celltype'] = 'IEEE4ByteReal'
    if not header.has_key('nullcellvalue'):
        header['nullcellvalue'] = '-99999'
    if not header.has_key('xdimension'):
        header['xdimension'] = '100'
    if not header.has_key('latitude'):
        header['latitude'] = '0:0:0'
    if not header.has_key('longitude'):
        header['longitude'] = '0:0:0'
    if not header.has_key('ydimension'):
        header['ydimension'] = '100'
    if not header.has_key('nroflines'):
        header['nroflines'] = '3'
    if not header.has_key('nrofcellsperline'):
        header['nrofcellsperline'] = '4'
    if not header.has_key('registrationcellx'):
        header['registrationcellx'] = '0'
    if not header.has_key('registrationcelly'):
        header['registrationcelly'] = str(int(header['nroflines'])-1)
    if not header.has_key('nrofbands'):
        header['nrofbands'] = '1'
    if not header.has_key('value'):
        header['value'] = '"Default_Band"'


    return header

In [6]:
print(3%0)

ZeroDivisionError: integer division or modulo by zero

In [1]:
import pyproj
print(pyproj.__version__)

2.1.0
